Computing analytic rank usinng Magma for CN curves with n upto 1,000,000

In [ ]:
from sage.all import *
from sage.parallel.decorate import parallel
import csv
import itertools
import time


INPUT_FILE = "sqfr1mil.csv"
OUTPUT_FILE = "an_rk_upto_1mil.csv"
CHUNK_SIZE = 50

Parallelism().set()  # Uses all available cores
print("Parallelism settings:", Parallelism().get_all())

magma = Magma()

@parallel(ncpus=15)  # Adjust ncpus to match your machine
def process_rows_parallel(chunk):
    results = []
    for row in chunk:
        try:
            n = Integer(row[0])
            a4 = -(n)**2  # Already minimal for squarefree n
            E = EllipticCurve([a4,0])
            an_rk_all = E.analytic_rank(algorithm='all')
            an_rk_zero_sum = E.analytic_rank(algorithm='zero_sum')

            magma_code = f'''
                E := EllipticCurve([{0}, {0}, {0}, {a4}, {0}]);
                AnalyticRank(E);
                '''
            #print(magma.eval(magma_code))
            an_rk_magma, sp_val_magma = magma.eval(magma_code).split()
            #results.append(row[0:] + [selmer2_rank] + [selmer3_rank])
            results.append(row[0:] + [an_rk_all] + [an_rk_zero_sum] + [an_rk_magma] + [sp_val_magma] )
        except Exception as e:
            print(f"Error processing n={row[0]}: {e}")
    return results

def stream_process_csv():
    with open(INPUT_FILE, newline='') as infile:
        reader = csv.reader(infile)
        header = next(reader)
        data_iter = iter(reader)

        # Initialize output file with header if it doesn't exist
        if not os.path.exists(OUTPUT_FILE):
            with open(OUTPUT_FILE, 'w', newline='') as outfile:
                writer = csv.writer(outfile)
                writer.writerow(header[0:] + ['an_rk_all'] + ['an_rk_zero_sum'] + ['an_rk_magma'] + ['sp_val_magma'])

        while True:
            chunk = list(itertools.islice(data_iter, CHUNK_SIZE))
            if not chunk:
                break

            for _, result in process_rows_parallel([chunk]):
                with open(OUTPUT_FILE, 'a', newline='') as outfile:
                    writer = csv.writer(outfile)
                    for row in result:
                        writer.writerow(row)

# Run main processing
stream_process_csv()

Parallelism settings: {'tensor': 1, 'linbox': 1}
